In [1]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

> A PyTorch computational graph is a Directed Acyclic Graph (DAG) that represents the sequence of operations performed on tensors during the forward pass of a model. This graph is dynamically built during runtime, meaning it is constructed on the fly as operations are executed.

In [2]:
x = torch.tensor(3.0, requires_grad=True, device=device) # Enables the differentation wrt this variable
x # A scalar

tensor(3., device='cuda:0', requires_grad=True)

In [3]:
y = x**2
y

tensor(9., device='cuda:0', grad_fn=<PowBackward0>)

> `backward` - Computes the gradient of current tensor wrt graph leaves. The graph is differentiated using the chain rule.

In [4]:
y.backward(retain_graph = True)
# Computational Graph: X -> square function -> y(leaf node)

In [5]:
x.grad
# Differentation of leaf node wrt x

tensor(6., device='cuda:0')

<u>Note</u>: When you run `y.backward()` again (only if `retain_graph = True`) then PyTorch saves the gradient in its memory and add it when you run it again.

In [6]:
x = torch.tensor(4.0, requires_grad=True)

y = 2 * x * (torch.cos(x) ** 2) # Multiplication Rule of Derivative

z = torch.sin(y)

In [7]:
x

tensor(4., requires_grad=True)

In [8]:
y

tensor(3.4180, grad_fn=<MulBackward0>)

In [9]:
z

tensor(-0.2729, grad_fn=<SinBackward0>)

> `Computational Graph`: X  →  func(2 * x * cos(x))  →  y  →  func(sin(y))  →  z

In [10]:
z.backward()

In [11]:
x.grad

tensor(6.7924)

In [12]:
y.grad
# You only able to calculate the gradient wrt leaf node (the last variable) only

C:\Users\shrir\AppData\Local\Temp\ipykernel_9940\896293680.py:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\build\aten\src\ATen/core/TensorBody.h:494.)
  y.grad


> Gradients of non leaf node will not be calculated

---

In [13]:
# Inputs
x = torch.tensor(6.7)  # Input feature
y = torch.tensor(0.0)  # True label (binary)

w = torch.tensor(1.0)  # Weight
b = torch.tensor(0.0)  # Bias

In [14]:
# Binary Cross-Entropy Loss for scalar
def binary_cross_entropy_loss(prediction, target):
    epsilon = 1e-8  # To prevent log(0)
    prediction = torch.clamp(prediction, epsilon, 1 - epsilon)
    return -(target * torch.log(prediction) + (1 - target) * torch.log(1 - prediction))

In [15]:
# Forward pass
z = w * x + b  # Weighted sum (linear part)
y_pred = torch.sigmoid(z)  # Predicted probability

# Compute binary cross-entropy loss
loss = binary_cross_entropy_loss(y_pred, y)
loss

tensor(6.7012)

In [16]:
# Derivatives:
# 1. dL/d(y_pred): Loss with respect to the prediction (y_pred)
dloss_dy_pred = (y_pred - y)/(y_pred*(1-y_pred))

# 2. dy_pred/dz: Prediction (y_pred) with respect to z (sigmoid derivative)
dy_pred_dz = y_pred * (1 - y_pred)

# 3. dz/dw and dz/db: z with respect to w and b
dz_dw = x  # dz/dw = x
dz_db = 1  # dz/db = 1 (bias contributes directly to z)

# Chain Rule
dL_dw = dloss_dy_pred * dy_pred_dz * dz_dw
dL_db = dloss_dy_pred * dy_pred_dz * dz_db

In [17]:
print(f"Manual Gradient of loss w.r.t weight (dw): {dL_dw}")
print(f"Manual Gradient of loss w.r.t bias (db): {dL_db}")

Manual Gradient of loss w.r.t weight (dw): 6.691762447357178
Manual Gradient of loss w.r.t bias (db): 0.998770534992218


In [18]:
x = torch.tensor(6.7)
y = torch.tensor(0.0)

w = torch.tensor(1.0, requires_grad=True)
b = torch.tensor(0.0, requires_grad=True)

In [19]:
# Forward Pass
z = w*x + b
z

tensor(6.7000, grad_fn=<AddBackward0>)

In [20]:
# Predict Probabilities
y_pred = torch.sigmoid(z)
y_pred

tensor(0.9988, grad_fn=<SigmoidBackward0>)

In [21]:
loss = binary_cross_entropy_loss(y_pred, y)
loss

tensor(6.7012, grad_fn=<NegBackward0>)

> `Computational Graph`: w, b  →  foreard pass(x, w, b)  →  z  →  probabilities(z)  →  y_pred  →  loss(y_pred, y_actual)

In [22]:
loss.backward()

In [23]:
print(w.grad)
print(b.grad)

tensor(6.6918)
tensor(0.9988)


---

In [24]:
# Gradients with multiple X values
x = torch.tensor([[1.0, 2.0, 3.0], [2.5, 9.8, 5.7]], requires_grad = True)
x

tensor([[1.0000, 2.0000, 3.0000],
        [2.5000, 9.8000, 5.7000]], requires_grad=True)

In [25]:
y = (x**2).mean()
y

tensor(24.7967, grad_fn=<MeanBackward0>)

In [26]:
y.backward()

In [27]:
x.grad

tensor([[0.3333, 0.6667, 1.0000],
        [0.8333, 3.2667, 1.9000]])

---

In [28]:
# clearing grad
x = torch.tensor(2.0, requires_grad = True)
x

tensor(2., requires_grad=True)

In [29]:
y = x ** 2
y

tensor(4., grad_fn=<PowBackward0>)

In [30]:
y.backward()

In [31]:
x.grad

tensor(4.)

> The problem with executing `y.backward()` and `x.grad` muptiple times will store the gradient values in the memory → If you run this more than one time then ouptut will be `d1 + d2 + ...` - The values of gradients will get added

In [32]:
# Inplace operation
x.grad.zero_()

tensor(0.)

---

## **<u>Disable gradient tracking</u>**

In [33]:
x = torch.tensor(2.0, requires_grad=True)
x

tensor(2., requires_grad=True)

In [34]:
y = x ** 2
y

tensor(4., grad_fn=<PowBackward0>)

In [35]:
y.backward()

In [36]:
x.grad

tensor(4.)

> 1. requires_grad_(False)
> 2. detach()
> 3. torch.no_grad()

In [37]:
x.requires_grad_(False) # requires_grad = False (_ inplace)

tensor(2.)

In [38]:
x

tensor(2.)

In [39]:
y = x ** 2

In [40]:
y

tensor(4.)

In [41]:
try:
    y.backward()
except Exception as e:
    print(e)

element 0 of tensors does not require grad and does not have a grad_fn


In [42]:
x = torch.tensor(2.0, requires_grad=True)
x

tensor(2., requires_grad=True)

In [43]:
y = x ** 2

In [44]:
y

tensor(4., grad_fn=<PowBackward0>)

In [45]:
y.backward()
x.grad

tensor(4.)

In [46]:
z = x.detach()
z

tensor(2.)

In [47]:
y = z ** 2
y

tensor(4.)

In [48]:
try:
    y.backward()
except Exception as e:
    print(e)

element 0 of tensors does not require grad and does not have a grad_fn


In [49]:
# Consider you are training your neural network
x = torch.tensor([1.0, 2.0], requires_grad=True) # Only Tensors of floating point and complex dtype can require gradients
x = torch.tensor(2.4, requires_grad=True)
y = x ** 2
y.backward()
print(x.grad)
# The neural network is trained

# Its prediction time
with torch.no_grad():
    # In this scope only, the x requires_grad parameter becomes false
    y = x + 10

    try:
        y.backward()
        print(x.grad())
    except Exception as e:
        print(e)

tensor(4.8000)
element 0 of tensors does not require grad and does not have a grad_fn


> `torch.no_grad()` is the most recommended way to stop calculating or tracking grad